In [1]:
import numpy as np
import torch
import cv2
import os
import random
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

from hydra.utils import instantiate
from hydra import compose, initialize_config_module 
from argparse import ArgumentParser

In [2]:
%cd /home/samba/320281459/code/sam2

/home/samba/320281459/code/sam2


In [ ]:
!pip3 install torch torchvision torchdata tensordict-nightly
!pip3 install tensorboard
!pip3 install fvcore
!pip install -e .

In [3]:
def add_pythonpath_to_sys_path():
    if "PYTHONPATH" not in os.environ or not os.environ["PYTHONPATH"]:
        return
    sys.path = os.environ["PYTHONPATH"].split(":") + sys.path

In [7]:
from training.utils.train_utils import makedir, register_omegaconf_resolvers

num_gpus=1
num_nodes=1
use_cluster=None

args_config = 'configs/sam2.1_training/sam2.1_hiera_l_Cardio_finetune.yaml'

register_omegaconf_resolvers()

ValueError: resolver 'get_method' is already registered

In [8]:
from hydra import compose
from omegaconf import OmegaConf
from iopath.common.file_io import g_pathmgr

cfg = compose(config_name=args_config)
# print(OmegaConf.to_yaml(cfg))
# instantiate(cfg.trainer, _recursive_=False)
if cfg.launcher.experiment_log_dir is None:
    cfg.launcher.experiment_log_dir = os.path.join(
        os.getcwd(), "sam2_logs", args_config
    )

add_pythonpath_to_sys_path()
makedir(cfg.launcher.experiment_log_dir)
with g_pathmgr.open(
    os.path.join(cfg.launcher.experiment_log_dir, "config.yaml"), "w"
) as f:
    f.write(OmegaConf.to_yaml(cfg))

cfg_resolved = OmegaConf.to_container(cfg, resolve=False)
cfg_resolved = OmegaConf.create(cfg_resolved)

with g_pathmgr.open(
    os.path.join(cfg.launcher.experiment_log_dir, "config_resolved.yaml"), "w"
) as f:
    f.write(OmegaConf.to_yaml(cfg_resolved, resolve=True))

submitit_conf = cfg.get("submitit", None)
assert submitit_conf is not None, "Missing submitit config"

submitit_dir = cfg.launcher.experiment_log_dir
submitit_dir = os.path.join(submitit_dir, "submitit_logs")
# Priotrize cmd line args
cfg.launcher.gpus_per_node = (
    num_gpus if num_gpus is not None else cfg.launcher.gpus_per_node
)
cfg.launcher.num_nodes = (
    num_nodes if num_nodes is not None else cfg.launcher.num_nodes
)
submitit_conf.use_cluster = (
    use_cluster if use_cluster is not None else submitit_conf.use_cluster
)

cfg.launcher.num_nodes = 1
main_port = random.randint(
    submitit_conf.port_range[0], submitit_conf.port_range[1]
)

num_proc = cfg.launcher.gpus_per_node
torch.multiprocessing.set_start_method(
    "spawn"
)  # CUDA runtime does not support `fork`

In [48]:
val_dataset = instantiate(cfg.trainer.data.get('val'))
dataloader = val_dataset.get_loader(epoch=0)

Raw dataset length = 1342
INFO 2025-03-26 12:17:31,422 sam2_datasets.py: 125: Dataset mixing probabilities: [1.0]


In [49]:
for i in dataloader:
    print(i)
    break

BatchedVideoDatapoint(
    dict_key=NonTensorData(data=val, batch_size=torch.Size([1]), device=None),
    img_batch=Tensor(shape=torch.Size([1, 20, 3, 512, 512]), device=cpu, dtype=torch.float32, is_shared=False),
    masks=Tensor(shape=torch.Size([1, 50, 512, 512]), device=cpu, dtype=torch.bool, is_shared=False),
    metadata=BatchedVideoMetaData(
        frame_orig_size=Tensor(shape=torch.Size([1, 50, 2]), device=cpu, dtype=torch.int64, is_shared=False),
        unique_objects_identifier=Tensor(shape=torch.Size([1, 50, 3]), device=cpu, dtype=torch.int64, is_shared=False),
        batch_size=torch.Size([1]),
        device=None,
        is_shared=False),
    obj_to_frame_idx=Tensor(shape=torch.Size([1, 50, 2]), device=cpu, dtype=torch.int32, is_shared=False),
    batch_size=torch.Size([1]),
    device=None,
    is_shared=False)


In [9]:
local_rank=0
world_size=num_proc

os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = str(main_port)
os.environ["RANK"] = str(local_rank)
os.environ["LOCAL_RANK"] = str(local_rank)
os.environ["WORLD_SIZE"] = str(world_size)

trainer = instantiate(cfg.trainer, _recursive_=False)

INFO 2025-03-26 11:47:07,153 train_utils.py: 108: MACHINE SEED: 4920
INFO 2025-03-26 11:47:07,157 train_utils.py: 154: Logging ENV_VARIABLES
INFO 2025-03-26 11:47:07,157 train_utils.py: 155: BROWSER=/home/samba/320281459/.vscode-server/cli/servers/Stable-ddc367ed5c8936efe395cffeec279b04ffd7db78/server/bin/helpers/browser.sh
CLICOLOR=1
CLICOLOR_FORCE=1
CONDA_DEFAULT_ENV=sam
CONDA_EXE=/home/samba/320281459/miniforge3/bin/conda
CONDA_PREFIX=/home/samba/320281459/miniforge3/envs/sam
CONDA_PROMPT_MODIFIER=(sam) 
CONDA_PYTHON_EXE=/home/samba/320281459/miniforge3/bin/python
CONDA_ROOT=/home/samba/320281459/miniforge3
CONDA_SHLVL=1
CUDA_MODULE_LOADING=LAZY
DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/31400531/bus
ELECTRON_RUN_AS_NODE=1
FORCE_COLOR=1
FTP_PROXY=http://146.112.255.50:80/
GIT_PAGER=cat
HOME=/home/samba/320281459
HTTPS_PROXY=http://146.112.255.50:443/
HTTP_PROXY=http://146.112.255.50:80/
LANG=en_US.UTF-8
LD_LIBRARY_PATH=/home/samba/320281459/miniforge3/envs/sam/lib/python3.11/site-

In [21]:
cfg.trainer.data

{'train': {'_target_': 'training.dataset.sam2_datasets.TorchTrainMixedDataset', 'phases_per_epoch': '${scratch.phases_per_epoch}', 'batch_sizes': ['${scratch.train_batch_size}'], 'datasets': [{'_target_': 'training.dataset.utils.RepeatFactorWrapper', 'dataset': {'_target_': 'training.dataset.utils.ConcatDataset', 'datasets': [{'_target_': 'training.dataset.vos_dataset.VOSDataset', 'transforms': '${vos.train_transforms}', 'training': True, 'video_dataset': {'_target_': 'training.dataset.vos_raw_dataset.PNGRawDataset', 'img_folder': '${dataset.train_img_folder}', 'gt_folder': '${dataset.train_gt_folder}', 'file_list_txt': '${dataset.file_list_txt}'}, 'sampler': {'_target_': 'training.dataset.vos_sampler.RandomUniformSampler', 'num_frames': '${scratch.num_frames}', 'max_num_objects': '${scratch.max_num_objects}'}, 'multiplier': '${dataset.multiplier}'}]}}], 'shuffle': True, 'num_workers': '${scratch.num_train_workers}', 'pin_memory': True, 'drop_last': True, 'collate_fn': {'_target_': 'tr

In [31]:
trainer.run_val()

/home/samba/320281459/code/sam2/training/trainer.py:626: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(


INFO 2025-03-26 11:55:12,470 train_utils.py: 271: Val Epoch: [0][  0/135] | Batch Time: 5.40 (5.40) | Data Time: 2.77 (2.77) | Mem (GB): 3.00 (3.00/3.00) | Time Elapsed: 00d 00h 08m | Losses/val_all_loss: 0.00e+00 (0.00e+00) | Losses/val_val_loss: 7.33e+00 (7.33e+00)
INFO 2025-03-26 11:55:28,446 train_utils.py: 271: Val Epoch: [0][ 10/135] | Batch Time: 1.43 (1.94) | Data Time: 0.00 (0.25) | Mem (GB): 3.00 (2.91/3.00) | Time Elapsed: 00d 00h 08m | Losses/val_all_loss: 0.00e+00 (0.00e+00) | Losses/val_val_loss: 7.49e+00 (7.34e+00)
INFO 2025-03-26 11:55:43,128 train_utils.py: 271: Val Epoch: [0][ 20/135] | Batch Time: 1.45 (1.72) | Data Time: 0.00 (0.13) | Mem (GB): 3.00 (2.90/3.00) | Time Elapsed: 00d 00h 08m | Losses/val_all_loss: 0.00e+00 (0.00e+00) | Losses/val_val_loss: 6.65e+00 (7.21e+00)
INFO 2025-03-26 11:55:58,223 train_utils.py: 271: Val Epoch: [0][ 30/135] | Batch Time: 1.56 (1.65) | Data Time: 0.00 (0.09) | Mem (GB): 3.00 (2.90/3.00) | Time Elapsed: 00d 00h 08m | Losses/val_a

In [8]:
model = trainer.model

trainable_parameters = sum(
    p.numel() for p in model.parameters(**{}) if p.requires_grad
)
total_parameters = sum(p.numel() for p in model.parameters(**{}))
non_trainable_parameters = total_parameters - trainable_parameters
print(trainable_parameters)
print(non_trainable_parameters)
model

80850178
0


DistributedDataParallel(
  (module): SAM2Train(
    (image_encoder): ImageEncoder(
      (trunk): Hiera(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(3, 112, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        )
        (blocks): ModuleList(
          (0): MultiScaleBlock(
            (norm1): LayerNorm((112,), eps=1e-06, elementwise_affine=True)
            (attn): MultiScaleAttention(
              (qkv): Linear(in_features=112, out_features=336, bias=True)
              (proj): Linear(in_features=112, out_features=112, bias=True)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((112,), eps=1e-06, elementwise_affine=True)
            (mlp): MLP(
              (layers): ModuleList(
                (0): Linear(in_features=112, out_features=448, bias=True)
                (1): Linear(in_features=448, out_features=112, bias=True)
              )
              (act): GELU(approximate='none')
            )
          )
          (1): Mul

In [ ]:
train